<a href="https://colab.research.google.com/github/maralthesage/visualizing_cnns/blob/main/Thesis_Visualization_Codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualizing CNNs

In this Notebook, two different methods of visualization are used to explain the model's prediction in CNN architectures VGG16, MobileNet and Inception V3, namely Grad-CAM and Occlusion Analysis. The codes for these two methods are adapted from the following resources:

* For Grad-CAM : https://keras.io/examples/vision/grad_cam/
* For Occlusion Analysis : https://deel-ai.github.io/xplique/api/attributions/occlusion/

First thing, we need to install the Xplique library for Occlusion Visualization and then importing the required libraries.

In [ ]:
!pip install -U -q xplique

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from IPython.display import Image, display

## --- Xplique tools --- ##
import xplique
from xplique.attributions import Occlusion
from xplique.plots import plot_attributions

%matplotlib inline

From the block below, choose the proper model name (from the given list) to visualize the results with respect to that certain model and mode of training it was trained on.

In [ ]:
## choose the name of models from the following list ==> 
## ['VGG-FT', 'VGG-FX', 'VGG-TR', 'MN-FT', 'MN-FX', 'MN-TR', 'IN-FT', 'IN-FX', 'IN-TR']
model_name = 'VGG-FT'
## models' paths are relative and the following address needs to be changed.
models_path = '/content/drive/MyDrive/Colab Notebooks/SomeFolder/'

# Model Import

At this section, the saved models are being imported from where you have them. This was originally addressed to Google Drive.

The following code block also sets the further setting required for the visualizations to work. 

**Please do NOT change anything in the following code**, but run it for the rest of the notebook to work properly. 


In [ ]:
### ------------------- Do Not Change This Code ------------------- ###
model = tf.keras.models.load_model(models_path+model_name)

if 'VGG' in model_name:
  IMG_SIZE = 224
  last_conv_layer = 'block5_conv3' 
  preprocess_input = tf.keras.applications.vgg16.preprocess_input

elif 'MN' in model_name:
  IMG_SIZE = 224
  last_conv_layer = 'conv_pw_13' 
  preprocess_input = tf.keras.applications.mobilenet.preprocess_input
  
elif 'IN' in model_name:
  IMG_SIZE = 299
  preprocess_input = tf.keras.applications.inception_v3.preprocess_input
  if 'FX' in model_name:
    last_conv_layer = 'conv2d_93' 
  elif 'FT' in model_name:
    last_conv_layer = 'conv2d_281' 
  elif 'TR' in model_name:
    last_conv_layer = 'conv2d_375' 



## Loading Images 

Here you need to change the `file_paths` to a path relative to where you keep the test data.

In [ ]:
## this path is relative to where you download and save the test_dataset file i shared as comment on top of this notebook
file_paths = '/content/drive/MyDrive/SomeFolder/data/'

img_paths = [
 f'{file_paths}Strawberry___healthy/15b5d300-a3f7-4e46-9f33-47f99c5d8364___RS_HL 4832.JPG',
 f'{file_paths}Strawberry___Leaf_scorch/d306834f-2362-49bc-aadb-8da5364d88f1___RS_L.Scorch 1334.JPG',
 f'{file_paths}Tomato___Leaf_Mold/528ee5e6-6c7f-4676-9fe2-fe3956a73b00___Crnl_L.Mold 6739.JPG',
 f'{file_paths}Peach___Bacterial_spot/316b51d9-ffb1-4ccd-ad84-62fe1d1d3e4d___Rut._Bact.S 1042.JPG',
 f'{file_paths}Apple___Cedar_apple_rust/f90bff66-b339-4a6f-acf4-85ed13aebff8___FREC_C.Rust 9843-horizontalflip.JPG',
 f'{file_paths}Tomato___Tomato_Yellow_Leaf_Curl_Virus/36f918e5-8c49-4820-a2fe-f3355e9f95e4___YLCV_NREC 2537.JPG',
 f'{file_paths}Potato___healthy/b925ad3e-fc49-497d-a6eb-115f0de20800___RS_HL 4170.JPG',
 f'{file_paths}Potato___Late_blight/d994fd7e-a338-42c5-ac37-ede01c18999e___RS_LB 5134.JPG',
 f'{file_paths}Tomato___healthy/4ed0a372-fe3d-49a6-82a1-2e904e6c66e3___GH_HL Leaf 200-horizontalflip.JPG',
 f'{file_paths}Corn_(maize)___Northern_Leaf_Blight/ccb8b8c4-840f-44b4-9a6e-43109f828b8c___RS_NLB 4091.JPG',
 f'{file_paths}Tomato___Target_Spot/e6c089fa-5b16-46e6-a8d9-f742377b43a8___Com.G_TgS_FL 8352.JPG',
 f'{file_paths}Apple___Apple_scab/fa9c1112-27fc-42db-930d-7e4956267ab5___FREC_Scab 3174.JPG', 
 f'{file_paths}Soybean___healthy/073d9dd0-012e-468d-96b9-494cb684d802___RS_HL 3563.JPG',
 f'{file_paths}Grape___Leaf_blight_(Isariopsis_Leaf_Spot)/e93f37ed-7dfd-401f-a470-0f932fecdd75___FAM_L.Blight 3637.JPG',
 f'{file_paths}Orange___Haunglongbing_(Citrus_greening)/08b7e039-7264-4a3d-b7d6-6e13c5bbac56___CREC_HLB 7417.JPG',
 f'{file_paths}Squash___Powdery_mildew/67c186ab-1874-4500-ae6b-9a575c8dcb42___MD_Powd.M 0752.JPG',
 f'{file_paths}Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot/d1f96a7c-c108-45a4-82f6-ffe84b0d081f___RS_GLSp 9304.JPG',
 f'{file_paths}Cherry_(including_sour)___Powdery_mildew/84ff1c46-5c01-4136-bef5-59cd2d0ed812___FREC_Pwd.M 0572.JPG',
 f'{file_paths}Corn_(maize)___Common_rust_/RS_Rust 1869.JPG',
 f'{file_paths}Pepper,_bell___Bacterial_spot/e05d9bf6-8d96-4fb5-8ea4-b9bab9e9bb8f___JR_B.Spot 3142.JPG',
 f'{file_paths}Pepper,_bell___healthy/2d117ef0-5705-4814-b191-1d184204452f___JR_HL 7744.JPG',
 f'{file_paths}Tomato___Late_blight/121097dd-b0cb-436b-9b2f-7dab30c86872___GHLB_PS Leaf 37.1 Day 13.jpg']

### Class Names

For printing purposes of the predictions and labels. 

In [ ]:
class_names = ['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy']

### Functions for preparing Images and Labels

The `get_img_array` function is adopted from Grad-CAM resource in https://keras.com. The rest are for name cleaning purposes. Just running the following would be enough.

In [ ]:

def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = tf.keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    array = np.expand_dims(array, axis=0)
    return array

## ---------------------------------------------- ##
## Clean up the class name for visualization use

def clean_name(filename):
  splitted_f = filename.split('_')
  name_pred = " ".join(word for word in splitted_f if word)
  return name_pred

## ---------------------------------------------- ##
## Get image predicted class name from the model

def class_pred(filepath):
  img = preprocess_input(get_img_array(filepath, size=IMG_SIZE))
  # img = get_img_array(filepath,(224,224))
  preds = model.predict(img,verbose=0)
  name_pred = class_names[np.argmax(preds)]
  # name_pred = clean_name(name_pred)
  return name_pred

## ---------------------------------------------- ##
## Get image true label from the filename

def true_label(filepath):
  for name in class_names:
    if name in filepath:
      return name



# Occlusion Analysis

The following two blocks will visualize the above added images using Occlusion method. The second one may take a while on a CPU.

In [ ]:
def occlusion(img_path):

    x =  np.expand_dims(tf.keras.preprocessing.image.load_img(img_path, target_size=(IMG_SIZE,IMG_SIZE)),0)
    x =  np.array(x, dtype=np.float32) / 255.0

    
    y =  np.expand_dims(tf.keras.utils.to_categorical(class_names.index(true_label(img_path)), 38), 0)

    explainer = Occlusion(model, 
                          patch_size=(32, 32), patch_stride=(16, 16), 
                          batch_size=16, occlusion_value=0)

    # compute explanation by calling the explainer
    explanation = explainer.explain(x, y)
    plot_attributions(explanation, x, img_size=3.2, cmap='jet', cols=1, alpha=.7)


In [ ]:
for id, img_path in enumerate(img_paths):
    occlusion(img_path)

# Grad-CAM 

The following code blocks will visualize models through Grad-CAM. The code does not require any modification and running it given all the data is loaded properly should visualize the results.


In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]


    grads = tape.gradient(class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
def create_heatmaps(image_path=img_path, model=model):
  model.layers[-1].activation = None

  img_array = preprocess_input(get_img_array(img_path, size=(IMG_SIZE,IMG_SIZE)))

  heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer)

  return heatmap

In [ ]:
def save_and_display_gradcam(img_path, cam_path="cam.jpg", alpha=1,id=0):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = create_heatmaps(img_path, model=model)
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show();


### Creating heatmaps and visualizing Grad-CAM

The following code will print the true_label and predicted label as well as the softmax prediction of the model along with the Grad-CAM visualization. 

Note: Sometimes the `Pred Value` print as a large number and that is a bug that occur after running the heatmap code. In case that happens, only reloading the saved model from the top of this notebook will fix the problem.

In [ ]:
for id,img_path in enumerate(img_paths):
  image = preprocess_input(get_img_array(img_path, size=(IMG_SIZE,IMG_SIZE)))
  pred  = model.predict(image,verbose=0)
  heatmap = create_heatmaps(img_path,model)

  print(f"True Label: {true_label(img_path)}")
  print(f"Prediction: {class_names[pred.argmax()]}")
  print(f"Pred Value: {pred.max():.2%}")
  save_and_display_gradcam(img_path,id=id)